<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 1
<br>
<b>Deadline:</b> March 23, 2020 (Monday). 23:00
</div>

# Exercise 4.2. Convolutional networks. VGG-style network.

In the second part you need to train a convolutional neural network with an architecture inspired by a VGG-network [(Simonyan \& Zisserman, 2015)](https://arxiv.org/abs/1409.1556).

In [1]:
skip_training = True  # Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tools
import tests

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## FashionMNIST dataset

Let us use the FashionMNIST dataset. It consists of 60,000 training images of 10 classes: 'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'.

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Min-max scaling to [-1, 1]
])

trainset = torchvision.datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

# VGG-style network

Let us now define a convolution neural network with an architecture inspired by the [VGG-net](https://arxiv.org/abs/1409.1556):

<img src="vgg-style.png" width=600 style="float: left;">

The architecture:
* A block of three convolutional layers with:
    * 3x3 kernel
    * 16 output channels
    * one pixel zero-pading on both sides
    * 2d batch normalization after each convolutional layer
    * ReLU nonlinearity after each 2d batch normalization layer
* Max pooling layer with 2x2 kernel and stride 2.
* A block of three convolutional layers with:
    * 3x3 kernel
    * 32 output channels
    * one pixel zero-pading on both sides
    * 2d batch normalization after each convolutional layer
    * ReLU nonlinearity after each 2d batch normalization layer
* Max pooling layer with 2x2 kernel and stride 2.
* One convolutional layer with:
    * 3x3 kernel
    * 48 output channels
    * *no padding*
    * 2d batch normalization after the convolutional layer
    * ReLU nonlinearity after the 2d batch normalization layer
* One convolutional layer with:
    * 1x1 kernel
    * 32 output channels
    * *no padding*
    * 2d batch normalization after the convolutional layer
    * ReLU nonlinearity after the 2d batch normalization layer
* One convolutional layer with:
    * 1x1 kernel
    * 16 output channels
    * *no padding*
    * 2d batch normalization after the convolutional layer
    * ReLU nonlinearity after the 2d batch normalization layer
* Global average pooling (compute the average value of each channel across all the input locations):
    * 5x5 kernel (the input of the layer should be 5x5)
* A fully-connected layer with 10 outputs (no nonlinearity)

Notes:
* Batch normalization is expected to be right after a convolutional layer, before nonlinearity.
* We recommend that you check the number of modules with trainable parameters in your network.

In [88]:
class VGGNet(nn.Module):
    def __init__(self, n_channels=16):
        """
        Args:
          n_channels (int): Number of channels in the first convolutional layer. The number of channels in the following layers are the multipliers of n_channels. Hence, parameters of the layers to follow can be defined using this variable.
        """
        super(VGGNet, self).__init__()
        # YOUR CODE HERE
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 48, kernel_size = 3),
            nn.BatchNorm2d(48),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(48, 32, kernel_size = 1),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size = 1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        
        self.fc = nn.Linear(16, 10)
        #self.pool = nn.AvgPool2d(5*5,kernel_size = 5)
        #raise NotImplementedError()

    def forward(self, x, verbose=False):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
          verbose: True if you want to print the shapes of the intermediate variables.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        # YOUR CODE HERE
        x = F.max_pool2d(self.layer1(x), kernel_size = 2, stride = 2)
        x = F.max_pool2d(self.layer2(x), kernel_size = 2, stride = 2)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = F.avg_pool2d(x,5)
        x = x.view(-1,16)
        x = self.fc(x)
        
        return x

In [89]:
def test_VGGNet_shapes():
    net = VGGNet()
    net.to(device)

    # Feed a batch of images from the training data to test the network
    with torch.no_grad():
        images, labels = iter(trainloader).next()
        images = images.to(device)
        print('Shape of the input tensor:', images.shape)

        y = net(images, verbose=True)
        assert y.shape == torch.Size([trainloader.batch_size, 10]), f"Bad y.shape: {y.shape}"

    print('Success')

test_VGGNet_shapes()

Shape of the input tensor: torch.Size([32, 1, 28, 28])
Success


In [90]:
tests.test_vgg_net(VGGNet)

y: tensor([[ 8.0032,  8.0032,  8.0032,  8.0032,  8.0032, -8.0032, -8.0032, -8.0032,
         -8.0032, -8.0032]], grad_fn=<AddmmBackward>)
expected: tensor([ 8.0032,  8.0032,  8.0032,  8.0032,  8.0032, -8.0032, -8.0032, -8.0032,
        -8.0032, -8.0032])
Success


# Train the network

In [91]:
# This function computes the accuracy on the test dataset
def compute_accuracy(net, testloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

### Training loop

Your task is to implement the training loop. The recommended hyperparameters:
* Adam optimizer with learning rate 0.01.
* Cross-entropy loss. Note that we did not use softmax nonlinearity in the final layer of our network. Therefore, we need to use a loss function with log_softmax implemented, such as [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss).
* Number of epochs: 10

We recommend you to use function `compute_accuracy()` defined above to track the accaracy during training. The test accuracy should be above 0.87.

**Note: function `compute_accuracy()` sets the network into the evaluation mode which changes the way the batch statistics are computed in batch normalization. You need to set the network into the training mode (by calling `net.train()`) when you want to perform training.**

In [94]:
net = VGGNet()

In [95]:
# Implement the training loop in this cell
LearningRate = 0.01
Momentum = 0.9
Epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = LearningRate, momentum = Momentum)

net.train()

if not skip_training:
    # YOUR CODE HERE
    #raise NotImplementedError()
    for epoch in range(Epochs):
        loss_sum = 0.0
        #read data form trainloader
        for batch, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device),labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            loss_sum += loss.item()
            
            if batch % 100 == 99:
                print('[%d, %d]loss: %.03f' % (epoch + 1, batch+1, loss_sum/100))
                loss_sum = 0.0
    

[1, 100]loss: 1.571
[1, 200]loss: 0.844
[1, 300]loss: 0.702
[1, 400]loss: 0.597
[1, 500]loss: 0.578
[1, 600]loss: 0.497
[1, 700]loss: 0.492
[1, 800]loss: 0.479
[1, 900]loss: 0.426
[1, 1000]loss: 0.420
[1, 1100]loss: 0.422
[1, 1200]loss: 0.367
[1, 1300]loss: 0.389
[1, 1400]loss: 0.373
[1, 1500]loss: 0.381
[1, 1600]loss: 0.383
[1, 1700]loss: 0.357
[1, 1800]loss: 0.365
[2, 100]loss: 0.322
[2, 200]loss: 0.333
[2, 300]loss: 0.314
[2, 400]loss: 0.299
[2, 500]loss: 0.334
[2, 600]loss: 0.292
[2, 700]loss: 0.297
[2, 800]loss: 0.302
[2, 900]loss: 0.306
[2, 1000]loss: 0.305
[2, 1100]loss: 0.303
[2, 1200]loss: 0.298
[2, 1300]loss: 0.283
[2, 1400]loss: 0.290
[2, 1500]loss: 0.282
[2, 1600]loss: 0.261
[2, 1700]loss: 0.284
[2, 1800]loss: 0.295
[3, 100]loss: 0.249
[3, 200]loss: 0.261
[3, 300]loss: 0.252
[3, 400]loss: 0.238
[3, 500]loss: 0.267
[3, 600]loss: 0.257
[3, 700]loss: 0.285
[3, 800]loss: 0.248
[3, 900]loss: 0.267
[3, 1000]loss: 0.229
[3, 1100]loss: 0.277
[3, 1200]loss: 0.255
[3, 1300]loss: 0.24

In [96]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
if not skip_training:
    tools.save_model(net, '4_vgg_net.pth')
else:
    net = VGGNet()
    tools.load_model(net, '4_vgg_net.pth', device)

Do you want to save the model (type yes to confirm)? yes
Model saved to 4_vgg_net.pth.


In [97]:
# Compute the accuracy on the test set
accuracy = compute_accuracy(net, testloader)
print('Accuracy of the VGG net on the test images: %.3f' % accuracy)
assert accuracy > 0.87, "Poor accuracy ({:.3f})".format(accuracy)
print('Success')

Accuracy of the VGG net on the test images: 0.925
Success
